In [8]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import time
import requests
import random
import json
import concurrent.futures
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
from seleniumbase import SB
import asyncio
import nest_asyncio

nest_asyncio.apply()

HEADERS = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'priority': 'u=1, i',
    'referer': "https://www.capterra.com/categories/",
    'sec-ch-ua': '"Not;A=Brand";v="99", "Google Chrome";v="139", "Chromium";v="139"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36',
}


In [2]:
def get_with_retries(scraper, url, headers, retries=2, delay=5):
    """Attempts to fetch a URL with a specified number of retries upon failure."""
    for attempt in range(retries + 1):
        try:
            response = scraper.get(url=url, headers=headers)
            response.raise_for_status()  # Will raise an HTTPError for bad responses (4xx or 5xx)
            return response
        except (requests.exceptions.RequestException, cloudscraper.exceptions.CloudflareException) as e:
            print(f"Request to {url} failed on attempt {attempt + 1}: {e}")
            if attempt < retries:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print(f"All retries failed for {url}.")
                raise  # Re-raise the last exception if all retries fail

In [9]:


sb_context_manager = SB(uc=False, headless=False, maximize=True,
                        #cap_string=cap_string
                        )

sb = sb_context_manager.__enter__()


Exception ignored in: <generator object SB at 0x136fc6a10>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/seleniumbase/plugins/sb_manager.py", line 1321, in SB
    sb.tearDown()
  File "/opt/anaconda3/lib/python3.11/site-packages/seleniumbase/fixtures/base_case.py", line 16483, in tearDown
    page_actions._reconnect_if_disconnected(self.driver)
  File "/opt/anaconda3/lib/python3.11/site-packages/seleniumbase/fixtures/page_actions.py", line 1567, in _reconnect_if_disconnected
    driver.connect()
  File "/opt/anaconda3/lib/python3.11/site-packages/seleniumbase/undetected/__init__.py", line 521, in connect
    self.start_session()
  File "/opt/anaconda3/lib/python3.11/site-packages/seleniumbase/undetected/__init__.py", line 551, in start_session
    super().start_session(capabilities)
  File "/opt/anaconda3/lib/python3.11/site-packages/selenium/webdriver/remote/webdriver.py", line 356, in start_session
    response = self.execute(Command.NEW_SESSI

In [10]:
url = "https://www.capterra.com/categories/"
sb.activate_cdp_mode(url)

sb.sleep(4)


In [11]:
html = sb.get_page_source()
soup = BeautifulSoup(html, 'html.parser')

In [18]:

list_raw = soup.select_one("div[data-testid*='alphabetical-list']")
list_all = list_raw.select("li[data-testid*='group-list-item']")

In [36]:
categories_data = []
for list_item in list_all:
    category_name = list_item.select_one("a").get_text(strip=True)  
    category_link = list_item.select_one("a").get("href")
    category_link = "https://www.capterra.com" + category_link
    cloud_link = category_link + "?deployment=CLOUD_SAAS_WEB_BASED"
    categories_data.append({
        'category_name': category_name,
        'category_link': category_link,
        'cloud_link': cloud_link
    })

all_categories_df = pd.DataFrame(categories_data)

In [37]:
all_categories_df

,category_name,category_link,cloud_link
0,360 Degree Feedback,https://www.capterra.com/360-degree-feedback-s...,https://www.capterra.com/360-degree-feedback-s...
1,3D Architecture,https://www.capterra.com/3d-architecture-softw...,https://www.capterra.com/3d-architecture-softw...
2,3D CAD,https://www.capterra.com/3d-cad-software/,https://www.capterra.com/3d-cad-software/?depl...
3,3D Rendering,https://www.capterra.com/3d-rendering-software/,https://www.capterra.com/3d-rendering-software...
4,AB Testing,https://www.capterra.com/ab-testing-software/,https://www.capterra.com/ab-testing-software/?...
...,...,...,...
994,Worship,https://www.capterra.com/worship-software/,https://www.capterra.com/worship-software/?dep...
995,XDR (Extended Detection & Response),https://www.capterra.com/xdr-software/,https://www.capterra.com/xdr-software/?deploym...
996,Yard Management,https://www.capterra.com/yard-management-softw...,https://www.capterra.com/yard-management-softw...
997,Yoga Studio,https://www.capterra.com/yoga-studio-software/,https://www.capterra.com/yoga-studio-software/...


In [40]:
row = all_categories_df.iloc[0]
link = row['cloud_link']
sb.cdp.open(link)

sb.sleep(5)

html = sb.get_page_source()
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
page_raw = soup.select_one('div[data-test-id = "current-page-display"]').get_text(strip=True)
if page_raw:
    # print(f"Raw page text: '{page_raw}'")  # Debug print to see exact format
    # Extract numbers from text like "Page1of15" (no spaces)
    match = re.search(r'of(\d+)', page_raw, re.IGNORECASE)
    if match:
        last_page = int(match.group(1))
        print(f"Last page number: {last_page}")
    else:
        # Try alternative patterns in case format varies
        match = re.search(r'(\d+)$', page_raw)  # Get the last number in the string
        if match:
            last_page = int(match.group(1))
            print(f"Last page number (alternative): {last_page}")
        else:
            print(f"Could not parse page info from: {page_raw}")
else:
    print("No page display element found")

Raw page text: 'Page1of15'
Last page number: 15

Last page number: 15


In [45]:
match